# Train Networks

- Train SoftMax or Multi-BCE classifier for the EEG diagnosis classification
    - CAUEEG-task1 benchmark: Classification of **Normal**, **MCI**, and **Dementia** symptoms
    - CAUEEG-task2 benchmark: Classification of **Normal** and **Abnormal** symptoms
- `Weights and Biases` sweep is used for hyperparameter search

-----

## Load Packages

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Load some packages
import os
import json
from copy import deepcopy
import gc
import time

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import wandb

# custom package
from datasets.caueeg_dataset import *
from datasets.caueeg_script import *
from models import *
from train import *

In [ ]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): 
    print('cuda is available:', torch.cuda.get_device_name(0))
else: 
    print('cuda is unavailable.') 

-----

## Set the default configuration for building datatset

In [ ]:
cfg_data = {}
cfg_data['device'] = device
cfg_data['task'] = 'task1'
cfg_data['dataset_path'] = r'local/dataset/02_Curated_Data_220419/'
cfg_data['file_format'] = 'memmap'  # 'feather', 'memmap'
cfg_data['load_event'] = False
cfg_data['latency'] = 200 * 10      # 10 seconds
cfg_data['seq_length'] = 200 * 10  # 10 seconds
cfg_data['longer_seq_length'] = 200 * 10 * 6 # 60 seconds
cfg_data['crop_multiple'] = 4
cfg_data['EKG'] = 'O'
cfg_data['photic'] = 'X'
cfg_data['input_norm'] = 'dataset'  # 'datatset', 'datapoint', 'no'
cfg_data['awgn'] = 5e-2
cfg_data['mgn'] = 1e-4
cfg_data['awgn_age'] = 5e-2

if '3090' in torch.cuda.get_device_name(0):
    cfg_data['minibatch'] = 256
elif '2080' in torch.cuda.get_device_name(0):
    cfg_data['minibatch'] = 128
elif '1070' in torch.cuda.get_device_name(0):
    cfg_data['minibatch'] = 96

In [ ]:
cfg_data_temp = deepcopy(cfg_data)
_ = build_dataset_for_train(cfg_data_temp, verbose=True)

in_channels = cfg_data_temp['preprocess_train'](next(iter(_[0])))['signal'].shape[1]
out_dims = len(cfg_data_temp['class_label_to_name'])

del _

In [ ]:
pprint.pprint(cfg_data_temp, width=250)

-----

## Define Network Models

In [ ]:
cfg_common_model = {'in_channels': in_channels, 
                    'out_dims': out_dims, 
                    'seq_length': cfg_data['seq_length']}

cfg_model_pool = []

### 1D Tiny CNN

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Tiny-CNN'
cfg_model['generator'] = TinyCNN1D
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

### M5 model

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-M5'
cfg_model['generator'] = M5
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

### 1D ResNet variants

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-18'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BasicBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-50'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-101'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

### 1D ResNeXt variants

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-50'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 4
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-101'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

### 2D ResNet variants

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-18' # resnet-18 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = BasicBlock2D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-50' # resnet-50 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
    'hop_length': 100,
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

### 2D ResNeXt variants

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-50' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 4
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-101' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

#### CNN-Transformer

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-CNN-Transformer'
cfg_model['generator'] = CNNTransformer
cfg_model['fc_stages'] = 2
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256  #
cfg_model['n_encoders'] = 4  #
cfg_model['n_heads'] = 4     #
cfg_model['dropout'] = 0.2   #
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

#### Summarize the loaded models

In [ ]:
for cfg_model in cfg_model_pool:
    pprint.pp(cfg_model, width=150)
    print('\n' + '-' * 100 + '\n')

-----

## Default Configurations for Training

In [ ]:
# training configurations
cfg_train = {}
cfg_train['iterations'] = (300000 * 32) // cfg_data['minibatch']
cfg_train['num_history'] = 500
cfg_train['lr_scheduler_type'] = 'transformer_style'
cfg_train['warmup_steps'] = 10000
cfg_train['weight_decay'] = 1e-2
cfg_train['mixup'] = 0.0 # 0 for no usage
cfg_train['criterion'] = 'cross-entropy' # 'cross-entropy', 'multi-bce'

cfg_train['device'] = device
cfg_train['save_model'] = True
cfg_train['draw_result'] = True
cfg_train['watch_model'] = False


cfg_train['LR'] = 1e-3  ##################################################################################
cfg_train['iterations'] = 300  ##################################################################################
cfg_train['num_history'] = 30  ##################################################################################

-----

## Train

In [ ]:
sweep_data = {}
sweep_data['seq_length'] = {
    'values': [
        200 *  5, #  5 sec
        200 * 10, # 10 sec
        200 * 15, # 15 sec
        200 * 20, # 20 sec
        200 * 30, # 30 sec
              ],
}

sweep_data['EKG'] = {
    'values': ['O', 'X'],
}

sweep_data['photic'] = {
    'values': ['O', 'X'],
}

sweep_data['awgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['mgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.1,
}

sweep_data['awgn_age'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

In [ ]:
sweep_model = {}
sweep_model['model_index'] = { 
    'values': [i for i in range(len(cfg_model_pool))] 
}

sweep_model['fc_stages'] = { 
    'distribution': 'int_uniform',
    'min': 1,
    'max': 4,
}

sweep_model['use_age'] = { 
    'values': ['fc', 'conv', 'no']
}

sweep_model['base_channels'] = { 
    'values': [32, 64, 128]
}

sweep_model['dropout'] = {
    'values': [0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

# sweep_model['LR'] = {
#     'distribution': 'log_uniform',
#     'min': math.log(5e-5),
#     'max': math.log(2e-3)
# }

sweep_model['activation'] = {
    'values': ['relu', 'gelu', 'mish']
}

In [ ]:
sweep_train = {}
# sweep_train['iterations'] = {
#     'values' : [150000]
# }

sweep_train['lr_scheduler_type'] = {
    'values': [
        'constant_with_decay',
        'transformer_style',
        'cosine_decay_with_warmup_half',
        'cosine_decay_with_warmup_one_and_half',
        'cosine_decay_with_warmup_two_and_half',
        'linear_with_warmup',
    ]

}

sweep_train['search_multiplier'] = {
    'values': [1.5, 2.0, 5.0]
}

sweep_train['weight_decay'] = {
    'distribution' : 'log_uniform_values',
    'min': 1e-5,
    'max': 1e-1
}

sweep_train['mixup'] = {
    'values': [0, 0.1, 0.2, 0.3]
}

sweep_train['criterion'] = {
    'values': ['cross-entropy', 'multi-bce']
}

In [ ]:
sweep_config = {
    "entity": "ipis-mjkim",
    "name" : "my-sweep",
    "method": "random",
    "parameters": 
    {
        **sweep_data,
        **sweep_model,
        **sweep_train,
    }
}

sweep_id = wandb.sweep(sweep_config, project=f"caueeg-{cfg_data['task']}")

In [ ]:
def train():
    # initialize the wandb log
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id

    with wandb_run:
        # collect some garbages
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
        # load default config
        config = {}
        cfg_model = cfg_model_pool[wandb.config.model_index]
        for k, v in {**cfg_data,  **cfg_model, **cfg_train}.items():
            if k not in wandb.config:
                config[k] = v

        # to prevent callables from type-conversion to str
        wandb.config.update(config)
        for k, v in wandb.config.items():
            if k not in config:
                config[k] = v

        # build dataset
        train_loader, val_loader, test_loader, test_loader_longer = build_dataset_for_train(config)
        
        # train the model
        model = train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, 
                                 config['preprocess_train'], config['preprocess_test'])
        
        # release memory
        del model
        del train_loader, val_loader
        del test_loader, test_loader_longer
        del config
        
    time.sleep(60)

In [ ]:
wandb.agent(sweep_id, function=train, count=100)